In [ ]:
import earthaccess
import pandas as pd
import numpy as np
import pystac_client
import odc.stac
import xarray as xr
import rasterio
from rasterio.env import Env
import os
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from rasterio.transform import xy
from pyproj import Transformer
import rioxarray as rxr
import re
from datetime import datetime

# Import Fmask functions from fmask module
%run module/fmask.ipynb

def list_all_s3_files_anonymous(bucket_name, prefix=None):
    """
    List all files from a public S3 bucket anonymously (no credentials needed).

    Args:
        bucket_name (str): The name of the public S3 bucket.
        prefix (str, optional): Filter by a folder or prefix path.

    Returns:
        list: List of S3 object keys (filenames).
    """
    s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))
    paginator = s3.get_paginator("list_objects_v2")
    kwargs = {"Bucket": bucket_name}
    if prefix:
        kwargs["Prefix"] = prefix

    files = []
    for page in paginator.paginate(**kwargs):
        files.extend([obj["Key"] for obj in page.get("Contents", [])])
    return files



def raster_center_latlon(path):
    da = rxr.open_rasterio(path)
    h, w = da.rio.height, da.rio.width
    x, y = da.rio.transform() * (w/2, h/2)
    return Transformer.from_crs(da.rio.crs, "EPSG:4326", always_xy=True).transform(x, y)



def search_cmr_stac(start_year,end_year,lat_range,lon_range,date_begin=None,date_end=None,mgrs_tile_id=None):
    
    """
    Searches the CMR STAC API for HLS granules within a specified spatial and temporal range.

    This function queries the HLSL30.v2.0 and HLSS30.v2.0 collections from NASA's LPCLOUD STAC endpoint.
    The search is split by year to handle API limitations and is most suitable for small to medium spatial domains.

    Parameters:
    -----------
    start_year : int or str
        The beginning year of the search period. Can be an integer (e.g., 2020) or a string (e.g., '2020-01-01').
    end_year : int or str
        The ending year of the search period (inclusive). Same format as start_year.
    lat_range : list or tuple
        A pair of latitude values specifying the search bounds, e.g., [min_lat, max_lat].
    lon_range : list or tuple
        A pair of longitude values specifying the search bounds, e.g., [min_lon, max_lon].
    date_begin : str, optional
        Optional start date (MM-DD) for each year, e.g., '06-01'. If None, defaults to '01-01'.
    date_end : str, optional
        Optional end date (MM-DD) for each year, e.g., '09-30'. If None, defaults to '12-31'.
    mgrs_tile_id : str, optional
        If provided, filters the returned items by the MGRS tile ID.

    Returns:
    --------
    items_list : list
        A flat list of STAC item objects (granules) that match the search criteria.
    """
    
    bbox = ([min(lon_range),min(lat_range),max(lon_range),max(lat_range)])
    if type(start_year) is int:
        years = np.arange(start_year,end_year+1,1)
    else:
        years = np.arange(int(start_year.split('-')[0]),
                          int(end_year.split('-')[0])+1
                          ,1)
    
    # Due to the limitation of the search, split into every year to perform the search
    # Note that this might work for a city scale, but for larger domain, the search can
    # even further needes to be constrained 
    items_list = list()
    for year in years:
        print(year)
        collections=['HLSL30.v2.0', 'HLSS30.v2.0']
        url='https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD'
        cloudcover_max=50
        lim=100
        if date_begin is not None:
            dt_min = str(year)+'-'+date_begin
            dt_max = str(year)+'-'+date_end
        else:
            dt_min = str(year)+'-01-01'
            dt_max = str(year)+'-12-31'
            
        # open the catalog
        catalog = pystac_client.Client.open(f'{url}')
        
        # perform the search
        search = catalog.search(
            collections=collections,
            bbox=bbox,
            datetime=dt_min + '/' + dt_max,
            limit=lim
        )
    
        items = list(search.items())

        if mgrs_tile_id is not None:
            items = [item for item in items if mgrs_tile_id in item.id]
                    
        
        items_list.append(items)
        print('Found', len(items), 'granules at point', bbox, 'from', dt_min, 'to', dt_max)

    
    items_list = [i for item in items_list for i in item]

    return items_list


def rename_common_bands(items_list):
    """
    Renames the spectral band asset keys in a list of HLS STAC items to a common naming convention.

    This function standardizes band names for both Sentinel-2 (HLS.S30) and Landsat (HLS.L30) items by 
    mapping their band-specific keys to a unified set of descriptive names: 
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR_1', 'SWIR_2'].

    For example:
    - HLS.S30 band 'B8A' → 'NIR'
    - HLS.L30 band 'B05' → 'NIR'

    Parameters:
    -----------
    items_list : list
        A list of STAC item objects containing HLS.L30 and/or HLS.S30 assets.

    Returns:
    --------
    items_list : list
        The same list of STAC items with band asset keys renamed to common names.
    """

    # Rename HLSS B8A and HLSL B05 to common band name
    S30_band_common = ['B02','B03','B04','B8A','B11','B12','B10']
    L30_band_common = ['B02','B03','B04','B05','B06','B07','B09']
    band_name = ['Blue','Green','Red','NIR','SWIR_1','SWIR_2','Cirrus']
    index=0
    for band in S30_band_common:
        
        for item in items_list:
            if "HLS.L30" in item.id:
                item.assets[band_name[index]] = item.assets.pop(L30_band_common[index])
            if "HLS.S30" in item.id:
                item.assets[band_name[index]] = item.assets.pop(band)
        
        index=index+1
            
    return items_list

def load_odc_stac(crs,bands,spatial_res,items_list,bbox):
    """
    Loads a spatiotemporal datacube from a list of STAC items using the ODC-STAC interface.

    This function sets the desired coordinate reference system (CRS), spatial resolution, and bounding box, 
    and loads the data lazily as an xarray Dataset with Dask chunking.

    Parameters:
    -----------
    crs : str or pyproj.CRS
        The target coordinate reference system for the output data (e.g., "EPSG:32614").
    
    bands : list of str
        List of band names (e.g., ['Blue', 'Green', 'Red']) to load from the STAC items.

    spatial_res : tuple of float
        The target spatial resolution as (x_resolution, y_resolution), e.g., (30, 30).

    items_list : list
        A list of STAC item objects (e.g., from pystac-client or search_cmr_stac) to be loaded.

    bbox : list or tuple
        Bounding box to constrain the spatial extent of the data, in the form [min_lon, min_lat, max_lon, max_lat].

    Returns:
    --------
    ds : xarray.Dataset
        A lazily-loaded xarray dataset containing the selected bands, spatially and temporally aligned.
    """

    # Set CRS and resolution, open lazily with odc.stac
    ds = odc.stac.stac_load(
        items_list,
        bbox=bbox,
        bands=(bands),
        crs=crs,
        resolution=spatial_res,
        #chunks=None,
        chunks={"band":1,"x":512,"y":512},  # If empty, chunks along band dim, 
        #groupby="solar_day", # This limits to first obs per day
    )

    return ds
    

def get_geometry_clip(city_name):
    """
    Retrieves the bounding box geometry of a specified urban area (city) from a US Census shapefile.

    This function loads a shapefile of Urban Areas (2018 vintage from the US Census),
    filters it by the given city name, extracts its bounding box, and returns it as a GeoDataFrame.

    Parameters:
    -----------
    city_name : str
        The name of the city to extract the bounding box for, matching the 'NAME10' field in the shapefile.

    Returns:
    --------
    df_geo_out : geopandas.GeoDataFrame
        A GeoDataFrame containing a single rectangular geometry (bounding box) for the specified city.
        The CRS matches that of the original shapefile.

    Notes:
    ------
    - The shapefile must be located at `'cb_2018_us_ua10_500k/cb_2018_us_ua10_500k.shp'` relative to the script.
    - The returned geometry is a bounding box, not the full city polygon.
    """
    from shapely.geometry import box
    import geopandas as gpd
    
    df_geo = gpd.read_file('cb_2018_us_ua10_500k/cb_2018_us_ua10_500k.shp')
    df_geo_loc = df_geo.loc[df_geo['NAME10'] == city_name]
    
    # create a bounding box from the shapefile 
    bounds = df_geo_loc.geometry.bounds.values[0]
    geom = box(*bounds)
    
    df_geo_out = gpd.GeoDataFrame({"id":1,"geometry":[box(*bounds)]})
    df_geo_out = df_geo_out.set_geometry('geometry')
    df_geo_out.crs = df_geo.crs

    return df_geo_out

def scale_hls_data(ds,bands):
    """
    Scales surface reflectance values in selected HLS bands to reflectance units.

    HLS data is provided as integer values scaled by a factor of 10,000. This function multiplies 
    the selected bands (excluding 'Fmask') by 0.0001 to convert them to reflectance values 
    in the range of approximately 0–1.

    Parameters:
    -----------
    ds : xarray.Dataset
        The dataset containing HLS bands as data variables.

    bands : list of str
        List of band names to scale (e.g., ['Blue', 'Green', 'Red', 'NIR']).
        Bands containing 'Fmask' in the name will be skipped.

    Returns:
    --------
    ds : xarray.Dataset
        The same dataset with the specified bands scaled to reflectance values.


    """

    for band in bands:

        if 'Fmask' not in band:
    
            ds[band].data = 0.0001 * ds[band].data

    return ds


def configure_gdal_rasterio_dask():
    """
    Configures GDAL, Rasterio, and Xarray to consistently use a custom GDAL environment 
    for reading remote datasets (e.g., cloud-hosted STAC assets).

    This function:
    1. Monkey-patches `xarray.Dataset.load()`, `xarray.DataArray.load()`, and 
       `xarray.Dataset.compute()` to ensure all data loading operations occur within 
       a custom GDAL environment using `rasterio.Env`.
    2. Monkey-patches `rasterio.open()` so any direct file access also respects this 
       GDAL configuration.

    The GDAL environment is customized with the following settings:
    - Enables reading cloud-optimized GeoTIFFs (COGs) over HTTP.
    - Uses a local cookie file (`~/cookies.txt`) for authenticated access.
    - Increases GDAL retry behavior for robustness against intermittent HTTP failures.
    - Disables directory reads on open to reduce HTTP overhead.

    These patches are helpful in workflows involving:
    - NASA's LP DAAC data accessed via STAC endpoints.
    - ODCE/Opendatacube-based STAC workflows using `odc.stac.stac_load()`.
    - Dask-enabled processing with remote assets and lazy loading.

    Dependencies:
    ------------
    - `xarray`
    - `rasterio`
    - `rasterio.Env` from `rasterio.env`
    """

    # ----------------- Step 1: 1. Monkey‑patch Xarray’s .load() to wrap every read in your Env ----------------- 
    # 1. Grab the real load method *before* patching
    _orig_ds_load = xr.Dataset.load
    _orig_da_load = xr.DataArray.load
    _orig_da_compute = xr.Dataset.compute
    
    def _load_with_env(self, **kwargs):
        # 2. In your Env you can set any GDAL/Rasterio opts
        with Env(
            GDAL_DISABLE_READDIR_ON_OPEN = "EMPTY_DIR",
            CPL_VSIL_CURL_ALLOWED_EXTENSIONS = "TIF",
            GDAL_HTTP_COOKIEFILE = '~/cookies.txt',
            GDAL_HTTP_COOKIEJAR = '~/cookies.txt',
            GDAL_HTTP_UNSAFESSL = 'YES',
            GDAL_HTTP_MAX_RETRY = '10',
            GDAL_HTTP_RETRY_DELAY = '0.5',
            CPL_VSIL_CURL_USE_HEAD = 'YES'
        ):
            # 3. Call the *original* load, not xr.Dataset.load
            return _orig_ds_load(self, **kwargs)
    
    def _da_load_with_env(self, **kwargs):
        with Env(
            GDAL_DISABLE_READDIR_ON_OPEN = "EMPTY_DIR",
            CPL_VSIL_CURL_ALLOWED_EXTENSIONS = "TIF",
            GDAL_HTTP_COOKIEFILE = '~/cookies.txt',
            GDAL_HTTP_COOKIEJAR = '~/cookies.txt',
            GDAL_HTTP_UNSAFESSL = 'YES',
            GDAL_HTTP_MAX_RETRY = '10',
            GDAL_HTTP_RETRY_DELAY = '0.5',
            CPL_VSIL_CURL_USE_HEAD = 'YES'
        ):
            return _orig_da_load(self, **kwargs)
            
    def _da_compute_with_env(self, **kwargs):
        with Env(
            GDAL_DISABLE_READDIR_ON_OPEN = "EMPTY_DIR",
            CPL_VSIL_CURL_ALLOWED_EXTENSIONS = "TIF",
            GDAL_HTTP_COOKIEFILE = '~/cookies.txt',
            GDAL_HTTP_COOKIEJAR = '~/cookies.txt',
            GDAL_HTTP_UNSAFESSL = 'YES',
            GDAL_HTTP_MAX_RETRY = '10',
            GDAL_HTTP_RETRY_DELAY = '0.5',
            CPL_VSIL_CURL_USE_HEAD = 'YES'
        ):
            return _orig_da_compute(self, **kwargs)
    
    # 4. Now monkey‑patch
    xr.Dataset.load   = _load_with_env
    xr.DataArray.load = _da_load_with_env
    xr.Dataset.compute = _da_compute_with_env
    # ----------------- Step 2: Monkey‑patch rasterio.open itself ----------------- 
    

    # 1. Capture the true open
    _orig_open = rasterio.open
    
    def open_with_env(*args, **kwargs):
        with Env(
            GDAL_DISABLE_READDIR_ON_OPEN = "EMPTY_DIR",
            CPL_VSIL_CURL_ALLOWED_EXTENSIONS = "TIF",
            GDAL_HTTP_COOKIEFILE = '~/cookies.txt',
            GDAL_HTTP_COOKIEJAR = '~/cookies.txt',
            GDAL_HTTP_UNSAFESSL = 'YES',
            GDAL_HTTP_MAX_RETRY = '10',
            GDAL_HTTP_RETRY_DELAY = '0.5',
            CPL_VSIL_CURL_USE_HEAD = 'YES'
        ):
            # 2. Call real open
            return _orig_open(*args, **kwargs)
    
    # 3. Replace it
    rasterio.open = open_with_env

def load_data_into_memory(ds_mask_scaled,split_para=1000):
    
    """
    Attempts to load a large xarray.Dataset into memory in chunks to avoid memory or I/O-related failures.

    If a full dataset `.load()` fails (e.g., due to size or remote data access limits), this function 
    falls back to iteratively loading spatial chunks along the x and y dimensions.

    Parameters:
    -----------
    ds_mask_scaled : xarray.Dataset
        A dataset with spatial dimensions ('x', 'y') that is lazily loaded (e.g., from a remote source or Dask).
    
    split_para : int, optional (default=1000)
        Defines the spatial chunk size used during fallback loading. Larger values load more data per attempt.

    Returns:
    --------
    ds_mask_scaled_sel : xarray.Dataset
        A dataset that has been loaded into memory, either in full or via spatial chunks.


    Example:
    --------
    >>> ds = odc.stac.stac_load(...)
    >>> ds_scaled = scale_hls_data(ds, bands)
    >>> ds_loaded = load_data_into_memory(ds_scaled, split_para=512)
    """
    
    max_attemp = int(len(ds_mask_scaled.x)/split_para)+2
    try:
        ds_mask_scaled_sel = ds_mask_scaled
        ds_mask_scaled_sel.load() 
    
    except:
        
        for i in range(max_attemp):
    
            if i==max_attemp:
                ds_mask_scaled_sel = ds_mask_scaled.sel(x=ds_mask_scaled.x[i:],
                                                    y=ds_mask_scaled.y[i:])
                
                ds_mask_scaled_sel = ds_mask_scaled_sel.chunk({'y':split_para*i/5,'x':split_para*i/5})
            else:
            
                ds_mask_scaled_sel = ds_mask_scaled.sel(x=ds_mask_scaled.x[i:split_para*i],
                                                    y=ds_mask_scaled.y[i:split_para*i])
    
                ds_mask_scaled_sel = ds_mask_scaled_sel.chunk({'y':split_para*i/5,'x':split_para*i/5})
            
            print(i,len(ds_mask_scaled_sel.x),len(ds_mask_scaled_sel.y))
            ds_mask_scaled_sel.load()

    return ds_mask_scaled_sel

In [ ]:


def download_fmask_for_mgrs_tiles(
    mrgs_ids_s3,
    files_s3,
    bucket_name,
    temp_dir,
    local_dir,
    buffer_lat=1.0,
    buffer_lon=1.0,
    bands=['Blue', 'Green', 'Red', 'Fmask', 'Cirrus']
):
    """
    Wrapper function to download HLS Fmask files from an S3 bucket 
    for each MGRS tile ID, based on temporal and spatial filters.

    Files are exported to:
        LOCAL_DIR / SAT_ID / MGRS_ID / filename

    Parameters
    ----------
    mrgs_ids_s3 : list
        List of MGRS tile IDs to process.
    files_s3 : list
        List of all available S3 file paths in the target bucket.
    bucket_name : str
        Name of the S3 bucket.
    temp_dir : str
        Temporary directory for intermediate storage.
    local_dir : str
        Base local directory for saving final files.
    buffer_lat, buffer_lon : float
        Spatial buffer (in degrees) around tile center.
    bands : list
        Bands to download (default includes 'Fmask').

    Returns
    -------
    downloaded_files : list
        List of local paths to downloaded files.
    """
    
    downloaded_files = []

    for mgrs_tile_id in mrgs_ids_s3:
        # Find matching S3 files for this tile
        files_s3_mgrs = [f for f in files_s3 if mgrs_tile_id in f]
        if not files_s3_mgrs:
            print(f"⚠️ No files found for {mgrs_tile_id}")
            continue
        
        # Convert file timestamp to Julian day
        days_s3 = np.unique([f.split('.')[3] for f in files_s3_mgrs])
        
        for day_julian in days_s3:
            # Determine satellite ID
            sat_id = 'S30' if 'S30' in files_s3_mgrs[0].split('/')[2] else 'L30'
            
            existing_files = glob.glob(os.path.join(local_dir+mgrs_tile_id+'/', f"HLS.{sat_id}.T{mgrs_tile_id}.{day_julian}*.tif"))               

        
        
            if len(existing_files) == 5:
                print(f"Skipping {mgrs_tile_id} in date {day_julian}: already processed.")
                continue
            
            print(f"🔹 Downloading for MGRS ID: {mgrs_tile_id}")
        
            # Get tile center
            raster_path = f"s3://{bucket_name}/{files_s3_mgrs[0]}"
            lon, lat = raster_center_latlon(raster_path)
        
            # Filter files based on the day_julian
            files_s3_mgrs_filter = [f for f in files_s3_mgrs if day_julian in f]
            

            
            # Sort files by date
            date_str = files_s3_mgrs_filter[0].split('.')[3]  # '2024006'
            start_year = date_str[:4]
            end_year = date_str[:4] 
            date_begin = datetime.strptime(date_str, "%Y%j").strftime("%m-%d")
            date_end = date_begin  # same for single file
            
            # Define AOI with buffer
            lat_range = (lat - buffer_lat, lat + buffer_lat)
            lon_range = (lon - buffer_lon, lon + buffer_lon)
            
            # Search STAC
            try:
                item_list = search_cmr_stac(start_year, end_year, lat_range, lon_range,
                                            date_begin, date_end, mgrs_tile_id=mgrs_tile_id)
            except Exception:
                item_list = search_cmr_stac(start_year, end_year, lat_range, lon_range,
                                            mgrs_tile_id=mgrs_tile_id)
            # Filter based on the sat_id
            item_list = [i for i in item_list if sat_id in i.id]
            
        
            
            if len(item_list) == 0 :
                print(f"⚠️ No matching STAC items found for {mgrs_tile_id} for this date {day_julian}")
                continue
            
            item_list_rename = rename_common_bands(item_list)
            
            # Create directory structure: LOCAL_DIR / SAT_ID / MGRS_ID
            output_dir = os.path.join(local_dir, mgrs_tile_id)
            os.makedirs(output_dir, exist_ok=True)
            
            # Download and save files
            for item in item_list_rename:
                for band in bands:
                    try:
                        download_url = item.assets[band].href
                        local_paths = earthaccess.download(download_url, local_path=output_dir)
                        downloaded_files.append(local_paths[0])
                    except Exception as e:
                        print(f"❌ Error downloading {band} for {mgrs_tile_id}: {e}")
            
            print(f"✅ Finished {mgrs_tile_id}: saved to {output_dir}\n")

    return downloaded_files


def load_fmask_pair(
    mrgs_id: str,
    bucket: str,
    fmask4_files: list[str] | str,
    fmask5_files: list[str] | str,
):
    """Load matching Fmask 4.7 and 5.0 images for a given MRGS ID from S3 as time-series DataArrays.

    Parameters
    ----------
    mrgs_id : str
        Granule (MRGS) identifier, e.g., ``'T20LKP'``.
    bucket : str
        Default S3 bucket name hosting the Fmask data. This is used when
        ``fmask4_files`` or ``fmask5_files`` are provided as prefix strings
        (not full S3 URIs).
    fmask4_files : list[str] | str
        Either (1) a pre-computed list of Fmask 4.7 object keys, (2) an S3 URI
        (e.g., ``'s3://bucket-name/path/to/folder/'``), or (3) a prefix string
        relative to the ``bucket`` parameter.
    fmask5_files : list[str] | str
        Same options as ``fmask4_files`` but for the Fmask 5.0 assets.

    Returns
    -------
    dict
        Dictionary with ``'fmask4'`` and ``'fmask5'`` entries containing xarray
        DataArrays with dimensions ``('time', 'y', 'x')``. The Fmask values are
        decoded to unique classification: 1 (water), 2 (cloud_shadow), 3 (snow_ice),
        4 (cloud), NaN (clear/other). Versions without data return ``None``.
        Each DataArray includes time coordinates extracted from filenames
        (Julian date format: YYYYDDD). Only time steps that exist in both
        Fmask 4.7 and 5.0 datasets are returned, ensuring aligned time series.
    """

    def _parse_s3_uri(uri: str) -> tuple[str, str]:
        trimmed = uri.strip()
        trimmed = trimmed[5:] if trimmed.startswith("s3://") else trimmed
        bucket_part, _, prefix_part = trimmed.partition('/')
        return bucket_part, prefix_part

    def _resolve_source(source, default_bucket):
        if isinstance(source, str):
            path = source.strip()
            if path.startswith("s3://"):
                bucket_name, prefix = _parse_s3_uri(path)
            else:
                bucket_name = default_bucket
                prefix = path.lstrip('/')
            files = list_all_s3_files_anonymous(bucket_name, prefix=prefix)
        else:
            # source is a list of file keys
            bucket_name = default_bucket
            files = source
        return bucket_name, files

    def _extract_time_from_filename(filename: str) -> str:
        """Extract Julian date (YYYYDDD) from HLS filename.
        
        Supports both HLS v2.0 and v5.0 filename formats:
        - v2.0: HLS.S30.T20LKP.2024006T105029.v2.0.Fmask.tif
        - v5.0: HLS.L30.T31UDQ.2024015.1
        """
        # Try v2.0 format first: 7 digits followed by 'T'
        match = re.search(r'\.(\d{7})T', filename)
        if match:
            return match.group(1)
        
        # Try v5.0 format: 7 digits followed by '.' or end of string
        match = re.search(r'\.(\d{7})(?:\.|$)', filename)
        if match:
            return match.group(1)
        
        return None

    def _find_all_keys(file_list, mrgs_id, pattern='Fmask.tif'):
        """Find all matching keys for the MRGS ID.
        
        Parameters
        ----------
        file_list : list
            List of file keys to search
        mrgs_id : str
            MRGS tile identifier
        pattern : str, optional
            Pattern to match in filename. Default is 'Fmask.tif'.
            Can be 'Fmask.tif', 'Red.tif', 'Green.tif', 'Blue.tif', etc.
        """
        matching_keys = []
        for key in file_list:
            if key.endswith('.tif') and f'{mrgs_id}.' in key and pattern in key:
                matching_keys.append(key)
        return sorted(matching_keys)  # Sort for consistent ordering
    
    def _find_rgb_keys(file_list, mrgs_id):
        """Find RGB band files (Red, Green, Blue) for the MRGS ID.
        
        Returns
        -------
        dict
            Dictionary with 'Red', 'Green', 'Blue' keys, each containing
            a list of matching file keys. Keys with no matches return empty list.
        """
        rgb_bands = {}
        # Common band name patterns
        band_patterns = {
            'Red': ['Red.tif', 'red.tif', 'B04.tif', 'B4.tif', 'Band4.tif'],
            'Green': ['Green.tif', 'green.tif', 'B03.tif', 'B3.tif', 'Band3.tif'],
            'Blue': ['Blue.tif', 'blue.tif', 'B02.tif', 'B2.tif', 'Band2.tif']
        }
        
        for band_name, patterns in band_patterns.items():
            matching = []
            for pattern in patterns:
                keys = _find_all_keys(file_list, mrgs_id, pattern=pattern)
                matching.extend(keys)
            rgb_bands[band_name] = sorted(list(set(matching)))  # Remove duplicates and sort
        
        return rgb_bands
    
    def _load_rgb_stack(rgb_keys_dict, bucket_name, common_times=None):
        """Load RGB bands and stack them along time dimension (optimized with parallel loading).
        
        Parameters
        ----------
        rgb_keys_dict : dict
            Dictionary with 'Red', 'Green', 'Blue' keys containing file lists
        bucket_name : str
            S3 bucket name
        common_times : list, optional
            List of datetime objects to filter time steps. If provided, only
            loads files matching these times.
        
        Returns
        -------
        dict or None
            Dictionary with 'Red', 'Green', 'Blue' keys, each containing
            an xarray.DataArray with dimensions (time, y, x). Returns None
            if no data available. Each band is loaded separately as a
            separate DataArray.
        """
        from concurrent.futures import ThreadPoolExecutor, as_completed
        
        def _load_single_raster(key, bucket_name, common_times):
            """Load a single raster file and return data with time."""
            s3_path = f"/vsis3/{bucket_name}/{key}"
            try:
                # Load as DataArray using rioxarray
                da = rxr.open_rasterio(s3_path)
                # Remove band dimension if present (assuming single band)
                if 'band' in da.dims:
                    da = da.squeeze('band', drop=True)
                
                # Extract date string from filename (e.g., '2024366' from '2024366T190105' or '2024358' from '2024358.1')
                time_str = _extract_time_from_filename(key)
                if time_str:
                    # Filter by common_times if provided - simple string comparison
                    if common_times is not None:
                        # Convert common_times to date strings (YYYYDDD format) for comparison
                        def extract_date_string(dt):
                            """Extract YYYYDDD date string from datetime object - simple conversion."""
                            import numpy as np
                            from datetime import date, datetime
                            
                            # Get a date object from various datetime types
                            if isinstance(dt, str):
                                # Already a string - extract first 7 digits if present
                                return dt[:7] if len(dt) >= 7 else dt
                            
                            # Convert to date object first
                            try:
                                if isinstance(dt, np.datetime64):
                                    dt = np.datetime64(dt, 'D').astype(object)
                                
                                if isinstance(dt, date):
                                    d = dt
                                elif isinstance(dt, datetime):
                                    d = dt.date()
                                elif hasattr(dt, 'date'):
                                    d = dt.date()
                                elif hasattr(dt, 'to_pydatetime'):
                                    d = dt.to_pydatetime().date()
                                elif hasattr(dt, 'item'):
                                    item = dt.item()
                                    if isinstance(item, date):
                                        d = item
                                    elif isinstance(item, datetime):
                                        d = item.date()
                                    elif isinstance(item, np.datetime64):
                                        d = np.datetime64(item, 'D').astype(object)
                                        if not isinstance(d, date):
                                            d = datetime.strptime(str(d)[:10], '%Y-%m-%d').date()
                                    else:
                                        # Try to parse as date string
                                        d = datetime.strptime(str(item)[:10], '%Y-%m-%d').date()
                                else:
                                    # Try to parse string representation
                                    d = datetime.strptime(str(dt)[:10], '%Y-%m-%d').date()
                                
                                # Format as YYYYDDD (Julian date)
                                return d.strftime('%Y%j')
                            except:
                                # Fallback: return first 7 characters of string representation
                                return str(dt)[:7] if len(str(dt)) >= 7 else str(dt)
                        
                        # Convert common_times to set of date strings (YYYYDDD format)
                        common_date_strings = set([extract_date_string(dt) for dt in common_times])
                        
                        # Simple string comparison - no datetime conversion needed
                        if time_str not in common_date_strings:
                            return None, None
                    
                    # Convert time string to datetime for return value (only when needed)
                    try:
                        time_dt = datetime.strptime(time_str, '%Y%j')
                        return da, time_dt
                    except ValueError:
                        return da, time_str
                else:
                    return da, None
            except Exception as e:
                print(f"Warning: Failed to load {key}: {e}")
                return None, None
        
        rgb_data = {}
        
        for band_name in ['Red', 'Green', 'Blue']:
            print(f"\n{'='*60}")
            print(f"Processing {band_name} band...")
            print(f"{'='*60}")
            keys = rgb_keys_dict.get(band_name, [])
            if not keys:
                print(f"  {band_name} band: No files found, skipping")
                continue
            
            # Filter keys to only those matching common_times if provided
            # (This filters to the pre-selected clear-sky images)
            if common_times is not None:
                # Convert common_times to date strings for matching
                def extract_date_string_simple(dt):
                    """Extract YYYYDDD date string from datetime object."""
                    import numpy as np
                    from datetime import date, datetime
                    if isinstance(dt, str):
                        return dt[:7] if len(dt) >= 7 else dt
                    try:
                        if isinstance(dt, np.datetime64):
                            dt = np.datetime64(dt, 'D').astype(object)
                        if isinstance(dt, date):
                            d = dt
                        elif isinstance(dt, datetime):
                            d = dt.date()
                        elif hasattr(dt, 'date'):
                            d = dt.date()
                        elif hasattr(dt, 'to_pydatetime'):
                            d = dt.to_pydatetime().date()
                        elif hasattr(dt, 'item'):
                            item = dt.item()
                            if isinstance(item, (date, datetime)):
                                d = item if isinstance(item, date) else item.date()
                            else:
                                d = datetime.strptime(str(item)[:10], '%Y-%m-%d').date()
                        else:
                            d = datetime.strptime(str(dt)[:10], '%Y-%m-%d').date()
                        return d.strftime('%Y%j')
                    except:
                        return str(dt)[:7] if len(str(dt)) >= 7 else str(dt)
                
                common_date_strings = set([extract_date_string_simple(dt) for dt in common_times])
                # Filter keys to only those matching the selected clear-sky times
                original_count = len(keys)
                filtered_keys = []
                for key in keys:
                    time_str = _extract_time_from_filename(key)
                    if time_str and time_str in common_date_strings:
                        filtered_keys.append(key)
                keys = filtered_keys
                if original_count > len(keys):
                    print(f"  {band_name} band: Filtered to {len(keys)} files matching selected clear-sky times (from {original_count} total)")
            
            if not keys:
                print(f"  {band_name} band: No files match the selected time steps, skipping")
                continue
            
            # Parallel loading using ThreadPoolExecutor
            dataarrays = []
            times = []
            
            with ThreadPoolExecutor(max_workers=3) as executor:
                # Submit all loading tasks
                future_to_key = {
                    executor.submit(_load_single_raster, key, bucket_name, common_times): key
                    for key in keys
                }
                
                # Collect results as they complete
                total_files = len(keys)
                loaded_count = 0
                skipped_count = 0
                processed_count = 0
                
                print(f"Loading {band_name} band: {total_files} files total")
                
                for future in as_completed(future_to_key):
                    da, time_val = future.result()
                    processed_count += 1
                    
                    if da is not None:
                        dataarrays.append(da)
                        if time_val is not None:
                            times.append(time_val)
                        else:
                            times.append(len(dataarrays) - 1)
                        loaded_count += 1
                    else:
                        skipped_count += 1
                    
                    # Progress tracking: show percentage and remaining
                    percentage = (processed_count / total_files) * 100
                    remaining = total_files - processed_count
                    print(f"  {band_name} band progress: {processed_count}/{total_files} ({percentage:.1f}%) - "
                          f"Loaded: {loaded_count}, Skipped: {skipped_count}, Remaining: {remaining}")
                


            
            if dataarrays:
                # Memory monitoring before concatenation
                try:
                    import psutil
                    import os
                    process = psutil.Process(os.getpid())
                    mem_before = process.memory_info().rss / 1024 / 1024  # MB
                    print(f"  {band_name} band: Starting concatenation of {len(dataarrays)} arrays...")
                    print(f"  {band_name} band: Memory before concatenation: {mem_before:.1f} MB")
                except ImportError:
                    print(f"  {band_name} band: Starting concatenation of {len(dataarrays)} arrays...")
                    print(f"  {band_name} band: (psutil not available for memory monitoring)")
                
                # Concatenate along time dimension using chunked approach for large datasets
                try:
                    print(f"  {band_name} band: Concatenating {len(dataarrays)} arrays...")
                    
                    # Use chunked concatenation for large datasets to avoid memory issues
                    chunk_size = 10  # Concatenate 10 arrays at a time
                    if len(dataarrays) > chunk_size:
                        print(f"  {band_name} band: Using chunked concatenation (chunk size: {chunk_size})")
                        chunks = []
                        time_chunks = []
                        
                        # Process in chunks
                        for i in range(0, len(dataarrays), chunk_size):
                            chunk_end = min(i + chunk_size, len(dataarrays))
                            chunk_arrays = dataarrays[i:chunk_end]
                            chunk_times = times[i:chunk_end]
                            
                            print(f"  {band_name} band: Concatenating chunk {i//chunk_size + 1}/{(len(dataarrays)-1)//chunk_size + 1} "
                                  f"(arrays {i+1}-{chunk_end})...")
                            
                            # Concatenate this chunk
                            chunk_concat = xr.concat(chunk_arrays, dim='time')
                            chunk_concat = chunk_concat.assign_coords(time=chunk_times)
                            chunks.append(chunk_concat)
                            time_chunks.extend(chunk_times)
                            
                            # Clear chunk from memory
                            del chunk_arrays, chunk_concat
                        
                        # Concatenate all chunks together
                        print(f"  {band_name} band: Concatenating {len(chunks)} chunks into final array...")
                        stacked = xr.concat(chunks, dim='time')
                        stacked = stacked.assign_coords(time=time_chunks)
                        
                        # Clear chunks from memory
                        del chunks
                    else:
                        # For small datasets, concatenate directly
                        stacked = xr.concat(dataarrays, dim='time')
                        stacked = stacked.assign_coords(time=times)
                    
                    print(f"  {band_name} band: Concatenation complete, assigning coordinates...")
                    print(f"  {band_name} band: Storing {band_name} band data...")
                    rgb_data[band_name] = stacked
                    
                    # Clear dataarrays from memory to free up space
                    del dataarrays
                    dataarrays = []  # Reset for next band if needed
                    
                    # Show data size information
                    try:
                        data_size_mb = stacked.nbytes / 1024 / 1024
                        print(f"  {band_name} band: Data size: {data_size_mb:.1f} MB, Shape: {stacked.shape}")
                    except:
                        pass
                    
                    # Memory monitoring after concatenation
                    try:
                        mem_after = process.memory_info().rss / 1024 / 1024  # MB
                        mem_used = mem_after - mem_before
                        print(f"  {band_name} band: Memory after concatenation: {mem_after:.1f} MB (used: {mem_used:.1f} MB)")
                    except:
                        pass
                    
                    print(f"  ✓ {band_name} band complete: {loaded_count} files loaded, {skipped_count} skipped")
                except MemoryError as e:
                    print(f"  ✗ {band_name} band: Memory error during concatenation: {e}")
                    print(f"  {band_name} band: Consider loading fewer files or using chunked processing")
                    raise
                except Exception as e:
                    print(f"  ✗ {band_name} band: Error during concatenation: {e}")
                    print(f"  {band_name} band: Error type: {type(e).__name__}")
                    raise
            else:
                print(f"  ✗ {band_name} band: No files loaded (all {skipped_count} files were skipped)")
        
        if not rgb_data:
            print("\nNo RGB data loaded - returning None")
            return None
        
        # Final summary
        print(f"\n{'='*60}")
        print("RGB loading complete!")
        print(f"{'='*60}")
        loaded_bands = list(rgb_data.keys())
        print(f"Successfully loaded bands: {loaded_bands}")
        for band_name in loaded_bands:
            try:
                band_size_mb = rgb_data[band_name].nbytes / 1024 / 1024
                print(f"  {band_name}: {rgb_data[band_name].shape}, {band_size_mb:.1f} MB")
            except:
                print(f"  {band_name}: {rgb_data[band_name].shape}")
        
        try:
            import psutil
            import os
            process = psutil.Process(os.getpid())
            total_mem = process.memory_info().rss / 1024 / 1024  # MB
            print(f"\nTotal memory usage: {total_mem:.1f} MB")
        except:
            pass
        
        # Return dictionary of separate DataArrays for each band
        return rgb_data

    def _load_raster_stack(keys, bucket_name):
        """Load multiple rasters, decode to unique bits, and stack them along time dimension."""
        if not keys:
            return None
        
        dataarrays = []
        times = []
        
        for key in keys:
            s3_path = f"/vsis3/{bucket_name}/{key}"
            try:
                # Load as DataArray using rioxarray
                da = rxr.open_rasterio(s3_path)
                # Remove band dimension if present (assuming single band)
                if 'band' in da.dims:
                    da = da.squeeze('band', drop=True)
                
                # Decode Fmask to unique classification values
                decoded = convert_fmask_unique(da)
                
                # Convert back to DataArray preserving coordinates and attributes
                da_decoded = xr.DataArray(
                    decoded,
                    dims=da.dims,
                    coords=da.coords,
                    attrs=da.attrs
                )
                
                # Extract time from filename
                time_str = _extract_time_from_filename(key)
                if time_str:
                    # Convert Julian date to datetime
                    try:
                        time_dt = datetime.strptime(time_str, '%Y%j')
                        times.append(time_dt)
                    except ValueError:
                        # Fallback: use the string as-is
                        times.append(time_str)
                else:
                    # Use index if time extraction fails
                    times.append(len(dataarrays))
                
                dataarrays.append(da_decoded)
            except Exception as e:
                print(f"Warning: Failed to load {key}: {e}")
                continue
        
        if not dataarrays:
            return None
        
        # Concatenate along time dimension
        stacked = xr.concat(dataarrays, dim='time')
        stacked = stacked.assign_coords(time=times)
        
        return stacked

    bucket4, fmask4_files = _resolve_source(fmask4_files, bucket)
    bucket5, fmask5_files = _resolve_source(fmask5_files, bucket)

    keys4 = _find_all_keys(fmask4_files, mrgs_id)
    keys5 = _find_all_keys(fmask5_files, mrgs_id)
    
    # Diagnostic output
    print(f"\n{'='*60}")
    print(f"Loading Fmask data for MRGS ID: {mrgs_id}")
    print(f"{'='*60}")
    print(f"Fmask 4.7: Found {len(keys4)} matching files in bucket '{bucket4}'")
    if len(keys4) > 0:
        print(f"  Sample files (first 3):")
        for key in keys4[:3]:
            print(f"    - {key}")
    else:
        print(f"  Searching in {len(fmask4_files)} total files")
        # Show sample of available files to help debug
        sample_files = [f for f in fmask4_files if mrgs_id in f][:5]
        if sample_files:
            print(f"  Sample files containing '{mrgs_id}' (first 5):")
            for f in sample_files:
                print(f"    - {f}")
        else:
            print(f"  No files found containing '{mrgs_id}'")
    
    print(f"Fmask 5.0: Found {len(keys5)} matching files in bucket '{bucket5}'")
    if len(keys5) > 0:
        print(f"  Sample files (first 3):")
        for key in keys5[:3]:
            print(f"    - {key}")
    else:
        print(f"  Searching in {len(fmask5_files)} total files")
        # Show sample of available files to help debug
        sample_files = [f for f in fmask5_files if mrgs_id in f][:5]
        if sample_files:
            print(f"  Sample files containing '{mrgs_id}' (first 5):")
            for f in sample_files:
                print(f"    - {f}")
        else:
            print(f"  No files found containing '{mrgs_id}'")
    print(f"{'='*60}\n")

    # Load both datasets
    fmask4_stack = _load_raster_stack(keys4, bucket4)
    fmask5_stack = _load_raster_stack(keys5, bucket5)
    
    # Match time steps: only return common time steps
    if fmask4_stack is not None and fmask5_stack is not None:
        # Find common time values
        time4 = set(fmask4_stack.time.values)
        time5 = set(fmask5_stack.time.values)
        common_times = sorted(list(time4.intersection(time5)))
        
        if len(common_times) == 0:
            print("Warning: No common time steps found between Fmask 4.7 and 5.0")
            return {
                'fmask4': None,
                'fmask5': None,
                'fmask4_red': None,
                'fmask4_green': None,
                'fmask4_blue': None,
            }
        
        # Select only common time steps
        fmask4_matched = fmask4_stack.sel(time=common_times)
        fmask5_matched = fmask5_stack.sel(time=common_times)
        
        print(f"Matched {len(common_times)} common time steps out of "
              f"{len(time4)} (Fmask 4.7) and {len(time5)} (Fmask 5.0) total time steps")
        
        # Helper function to select top N clear-sky images based on cloud coverage
        def select_clear_sky_images(fmask_data, n=10):
            """
            Select top N clear-sky images (lowest cloud coverage) from Fmask data.
            
            Parameters
            ----------
            fmask_data : xarray.DataArray
                Fmask DataArray with dimensions (time, y, x)
                Values: 1=water, 2=cloud_shadow, 3=snow_ice, 4=cloud, NaN=clear
            n : int
                Number of clear-sky images to select (default: 10)
            
            Returns
            -------
            list
                List of time values (datetime objects) for the top N clear-sky images
            """
            if fmask_data is None or 'time' not in fmask_data.dims:
                return []
            
            import numpy as np
            
            # Calculate cloud coverage percentage for each time step
            # Cloud pixels are where value == 4
            cloud_coverage = []
            time_values = []
            
            for time_val in fmask_data.time.values:
                time_slice = fmask_data.sel(time=time_val)
                
                # Count cloud pixels (value == 4)
                cloud_pixels = (time_slice == 4).sum().values
                
                # Count total valid (non-NaN) pixels
                valid_pixels = (~np.isnan(time_slice)).sum().values
                
                if valid_pixels > 0:
                    # Calculate cloud coverage percentage
                    coverage_pct = (cloud_pixels / valid_pixels) * 100.0
                    cloud_coverage.append(coverage_pct)
                    time_values.append(time_val)
                else:
                    # Skip time steps with no valid data
                    continue
            
            if not cloud_coverage:
                return []
            
            # Sort by cloud coverage (ascending - least clouds first)
            # Get indices sorted by cloud coverage
            sorted_indices = np.argsort(cloud_coverage)
            
            # Select top N clear-sky images
            n_selected = min(n, len(sorted_indices))
            selected_indices = sorted_indices[:n_selected]
            selected_times = [time_values[i] for i in selected_indices]
            selected_coverage = [cloud_coverage[i] for i in selected_indices]
            
            print(f"Selected top {n_selected} clear-sky images:")
            print(f"  Cloud coverage range: {min(selected_coverage):.2f}% - {max(selected_coverage):.2f}%")
            print(f"  Average cloud coverage: {np.mean(selected_coverage):.2f}%")
            
            return selected_times
        
        # Select top 10 clear-sky images from Fmask 4.7 for RGB loading
        clear_sky_times = select_clear_sky_images(fmask4_matched, n=10)
        
        # Load RGB bands for Fmask 4.7 if available (separate bands)
        # Only load RGB for the selected clear-sky images
        fmask4_rgb_dict = None
        try:
            rgb_keys = _find_rgb_keys(fmask4_files, mrgs_id)
            if any(rgb_keys.values()):  # Check if any RGB bands found
                if clear_sky_times:
                    print(f"Loading RGB bands for {len(clear_sky_times)} clear-sky images...")
                    fmask4_rgb_dict = _load_rgb_stack(rgb_keys, bucket4, common_times=clear_sky_times)
                else:
                    print("Warning: No clear-sky images found, skipping RGB loading")
                if fmask4_rgb_dict is not None:
                    loaded_bands = [k for k in ['Red', 'Green', 'Blue'] if k in fmask4_rgb_dict]
                    print(f"Loaded RGB bands for Fmask 4.7: {loaded_bands}")
        except Exception as e:
            print(f"Warning: Could not load RGB bands for Fmask 4.7: {e}")
        
        return {
            'fmask4': fmask4_matched,
            'fmask5': fmask5_matched,
            'fmask4_red': fmask4_rgb_dict.get('Red') if fmask4_rgb_dict else None,
            'fmask4_green': fmask4_rgb_dict.get('Green') if fmask4_rgb_dict else None,
            'fmask4_blue': fmask4_rgb_dict.get('Blue') if fmask4_rgb_dict else None,
        }

    elif fmask4_stack is not None:
        print("Warning: Fmask 5.0 data not available")
        
        # Helper function to select top N clear-sky images based on cloud coverage
        def select_clear_sky_images_fmask4_only(fmask_data, n=10):
            """
            Select top N clear-sky images (lowest cloud coverage) from Fmask data.
            
            Parameters
            ----------
            fmask_data : xarray.DataArray
                Fmask DataArray with dimensions (time, y, x)
                Values: 1=water, 2=cloud_shadow, 3=snow_ice, 4=cloud, NaN=clear
            n : int
                Number of clear-sky images to select (default: 10)
            
            Returns
            -------
            list
                List of time values (datetime objects) for the top N clear-sky images
            """
            if fmask_data is None or 'time' not in fmask_data.dims:
                return []
            
            import numpy as np
            
            # Calculate cloud coverage percentage for each time step
            cloud_coverage = []
            time_values = []
            
            for time_val in fmask_data.time.values:
                time_slice = fmask_data.sel(time=time_val)
                cloud_pixels = (time_slice == 4).sum().values
                valid_pixels = (~np.isnan(time_slice)).sum().values
                
                if valid_pixels > 0:
                    coverage_pct = (cloud_pixels / valid_pixels) * 100.0
                    cloud_coverage.append(coverage_pct)
                    time_values.append(time_val)
            
            if not cloud_coverage:
                return []
            
            sorted_indices = np.argsort(cloud_coverage)
            n_selected = min(n, len(sorted_indices))
            selected_indices = sorted_indices[:n_selected]
            selected_times = [time_values[i] for i in selected_indices]
            selected_coverage = [cloud_coverage[i] for i in selected_indices]
            
            print(f"Selected top {n_selected} clear-sky images:")
            print(f"  Cloud coverage range: {min(selected_coverage):.2f}% - {max(selected_coverage):.2f}%")
            print(f"  Average cloud coverage: {np.mean(selected_coverage):.2f}%")
            
            return selected_times
        
        # Select top 10 clear-sky images from Fmask 4.7 for RGB loading
        clear_sky_times = select_clear_sky_images_fmask4_only(fmask4_stack, n=10)
        
        # Try to load RGB bands (separate bands)
        # Only load RGB for the selected clear-sky images
        fmask4_rgb_dict = None
        try:
            rgb_keys = _find_rgb_keys(fmask4_files, mrgs_id)
            if any(rgb_keys.values()):
                if clear_sky_times:
                    print(f"Loading RGB bands for {len(clear_sky_times)} clear-sky images...")
                    fmask4_rgb_dict = _load_rgb_stack(rgb_keys, bucket4, common_times=clear_sky_times)
                else:
                    print("Warning: No clear-sky images found, skipping RGB loading")
                if fmask4_rgb_dict is not None:
                    loaded_bands = [k for k in ['Red', 'Green', 'Blue'] if k in fmask4_rgb_dict]
                    print(f"Loaded RGB bands for Fmask 4.7: {loaded_bands}")
        except Exception as e:
            print(f"Warning: Could not load RGB bands for Fmask 4.7: {e}")
        
        return {
            'fmask4': fmask4_stack,
            'fmask5': None,
            'fmask4_red': fmask4_rgb_dict.get('Red') if fmask4_rgb_dict else None,
            'fmask4_green': fmask4_rgb_dict.get('Green') if fmask4_rgb_dict else None,
            'fmask4_blue': fmask4_rgb_dict.get('Blue') if fmask4_rgb_dict else None,
        }
        
    elif fmask5_stack is not None:
        print("Warning: Fmask 4.7 data not available")
        return {
            'fmask4': None,
            'fmask5': fmask5_stack,
            'fmask4_red': None,
            'fmask4_green': None,
            'fmask4_blue': None,
        }
    else:
        print("Warning: No Fmask data available")
        return {
            'fmask4': None,
            'fmask5': None,
            'fmask4_red': None,
            'fmask4_green': None,
            'fmask4_blue': None,
        }
